# Llama2Chat

本笔记本展示了如何使用 `Llama2Chat` 包装器来增强 Llama-2 的 `LLM`，以支持 [Llama-2 聊天提示格式](https://huggingface.co/blog/llama2#how-to-prompt-llama-2)。LangChain 中有几个 `LLM` 实现可以用作 Llama-2 聊天模型的接口。这些包括 [ChatHuggingFace](/docs/integrations/chat/huggingface)、[LlamaCpp](/docs/integrations/chat/llamacpp/)、[GPT4All](/docs/integrations/llms/gpt4all) 等，仅举几例。

`Llama2Chat` 是一个通用的包装器，它实现了 `BaseChatModel`，因此可以作为 [聊天模型](/docs/how_to#chat-models) 在应用程序中使用。`Llama2Chat` 将消息列表转换为 [必需的聊天提示格式](https://huggingface.co/blog/llama2#how-to-prompt-llama-2)，并将格式化后的提示作为 `str` 转发给被包装的 `LLM`。

In [1]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat

对于下面的聊天应用程序示例，我们将使用以下聊天 `prompt_template`：

In [2]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

template_messages = [
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{text}"),
]
prompt_template = ChatPromptTemplate.from_messages(template_messages)

## 使用 `HuggingFaceTextGenInference` LLM 与 Llama-2 进行聊天

HuggingFaceTextGenInference LLM 封装了对 [text-generation-inference](https://github.com/huggingface/text-generation-inference) 服务器的访问。在下面的示例中，推理服务器服务的是 [meta-llama/Llama-2-13b-chat-hf](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf) 模型。可以使用以下命令在本地启动：

```bash
docker run \
  --rm \
  --gpus all \
  --ipc=host \
  -p 8080:80 \
  -v ~/.cache/huggingface/hub:/data \
  -e HF_API_TOKEN=${HF_API_TOKEN} \
  ghcr.io/huggingface/text-generation-inference:0.9 \
  --hostname 0.0.0.0 \
  --model-id meta-llama/Llama-2-13b-chat-hf \
  --quantize bitsandbytes \
  --num-shard 4
```

例如，这在配备 4 块 RTX 3080ti 显卡的机器上运行。请根据可用的 GPU 数量调整 `--num_shard` 的值。`HF_API_TOKEN` 环境变量用于存放 Hugging Face API 令牌。

In [ ]:
# !pip3 install text-generation

创建一个 `HuggingFaceTextGenInference` 实例，连接到本地推理服务器，并将其包装到 `Llama2Chat` 中。

In [4]:
from langchain_community.llms import HuggingFaceTextGenInference

llm = HuggingFaceTextGenInference(
    inference_server_url="http://127.0.0.1:8080/",
    max_new_tokens=512,
    top_k=50,
    temperature=0.1,
    repetition_penalty=1.03,
)

model = Llama2Chat(llm=llm)

那么，您就可以准备好将聊天 `model` 与 `prompt_template` 和对话 `memory` 一起在 `LLMChain` 中使用了。

In [5]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)

In [6]:
print(
    chain.run(
        text="What can I see in Vienna? Propose a few locations. Names only, no details."
    )
)

 Sure, I'd be happy to help! Here are a few popular locations to consider visiting in Vienna:

1. Schönbrunn Palace
2. St. Stephen's Cathedral
3. Hofburg Palace
4. Belvedere Palace
5. Prater Park
6. Vienna State Opera
7. Albertina Museum
8. Museum of Natural History
9. Kunsthistorisches Museum
10. Ringstrasse


In [7]:
print(chain.run(text="Tell me more about #2."))

 Certainly! St. Stephen's Cathedral (Stephansdom) is one of the most recognizable landmarks in Vienna and a must-see attraction for visitors. This stunning Gothic cathedral is located in the heart of the city and is known for its intricate stone carvings, colorful stained glass windows, and impressive dome.

The cathedral was built in the 12th century and has been the site of many important events throughout history, including the coronation of Holy Roman emperors and the funeral of Mozart. Today, it is still an active place of worship and offers guided tours, concerts, and special events. Visitors can climb up the south tower for panoramic views of the city or attend a service to experience the beautiful music and chanting.


## 通过 `LlamaCPP` LLM 与 Llama-2 聊天

要使用 Llama-2 chat 模型与 [LlamaCPP](/docs/integrations/llms/llamacpp) `LMM`，请按照[此处的安装说明](/docs/integrations/llms/llamacpp#installation)安装 `llama-cpp-python` 库。以下示例使用的是一个量化的 [llama-2-7b-chat.Q4_0.gguf](https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_0.gguf) 模型，该模型本地存储在 `~/Models/llama-2-7b-chat.Q4_0.gguf`。

创建 `LlamaCpp` 实例后，`llm` 会再次被包装进 `Llama2Chat`。

In [ ]:
from os.path import expanduser

from langchain_community.llms import LlamaCpp

model_path = expanduser("~/Models/llama-2-7b-chat.Q4_0.gguf")

llm = LlamaCpp(
    model_path=model_path,
    streaming=False,
)
model = Llama2Chat(llm=llm)

并在与上一个示例相同的方式中使用。

In [9]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)

In [10]:
print(
    chain.run(
        text="What can I see in Vienna? Propose a few locations. Names only, no details."
    )
)

  Of course! Vienna is a beautiful city with a rich history and culture. Here are some of the top tourist attractions you might want to consider visiting:
1. Schönbrunn Palace
2. St. Stephen's Cathedral
3. Hofburg Palace
4. Belvedere Palace
5. Prater Park
6. MuseumsQuartier
7. Ringstrasse
8. Vienna State Opera
9. Kunsthistorisches Museum
10. Imperial Palace

These are just a few of the many amazing places to see in Vienna. Each one has its own unique history and charm, so I hope you enjoy exploring this beautiful city!



llama_print_timings:        load time =     250.46 ms
llama_print_timings:      sample time =      56.40 ms /   144 runs   (    0.39 ms per token,  2553.37 tokens per second)
llama_print_timings: prompt eval time =    1444.25 ms /    47 tokens (   30.73 ms per token,    32.54 tokens per second)
llama_print_timings:        eval time =    8832.02 ms /   143 runs   (   61.76 ms per token,    16.19 tokens per second)
llama_print_timings:       total time =   10645.94 ms


In [11]:
print(chain.run(text="Tell me more about #2."))

Llama.generate: prefix-match hit


  Of course! St. Stephen's Cathedral (also known as Stephansdom) is a stunning Gothic-style cathedral located in the heart of Vienna, Austria. It is one of the most recognizable landmarks in the city and is considered a symbol of Vienna.
Here are some interesting facts about St. Stephen's Cathedral:
1. History: The construction of St. Stephen's Cathedral began in the 12th century on the site of a former Romanesque church, and it took over 600 years to complete. The cathedral has been renovated and expanded several times throughout its history, with the most significant renovation taking place in the 19th century.
2. Architecture: St. Stephen's Cathedral is built in the Gothic style, characterized by its tall spires, pointed arches, and intricate stone carvings. The cathedral features a mix of Romanesque, Gothic, and Baroque elements, making it a unique blend of styles.
3. Design: The cathedral's design is based on the plan of a cross with a long nave and two shorter arms extending from


llama_print_timings:        load time =     250.46 ms
llama_print_timings:      sample time =     100.60 ms /   256 runs   (    0.39 ms per token,  2544.73 tokens per second)
llama_print_timings: prompt eval time =    5128.71 ms /   160 tokens (   32.05 ms per token,    31.20 tokens per second)
llama_print_timings:        eval time =   16193.02 ms /   255 runs   (   63.50 ms per token,    15.75 tokens per second)
llama_print_timings:       total time =   21988.57 ms
